<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Time Series Forecasting on Number of Passengers for FlyHigh Airline</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Consider <b>FlyHigh Airlines</b> as our client. The client requires us to forecast the number of passengers who will fly their airline. The ARIMA model has been widely utilized in many fields for forecasting since it is recognized to be reliable, efficient, and capable of predicting short-term share market movements. Hence, we'll build an ARIMA model to forecast the demand(passenger traffic) in Airplanes.
<br>
<br>
ARIMA(Auto Regressive Integrated Moving Average) is a combination of 2 models AR(Auto Regressive) and MA(Moving Average). It has 3 hyperparameters - P(auto regressive lags), d(order of differentiation), Q(moving avg.) which respectively comes from the AR, I and MA components. The AR part is correlation between prev and current time periods. To smooth out the noise, the MA part is used. The I part binds together the AR and MA parts.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Data:</b></p>
<p style = 'font-size:16px;font-family:Arial'>The data for this demonstration resides on Vantage. The data consists of monthly flight totals by month between 1949 and 1960  (144 rows) which we selected to simply be able to understand the functionality. The same functionality can be applied to huge datasets with multiple series concurrently.
</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Connect to Vantage</li>    
    <li>Explore the dataset</li>
    <li>Check for Stationarity using Dickey-Fuller Test</li>
    <li>Make series stationary using TD_DIFF (D)</li>
    <li>Check for autocorrelation of the time series (Q)</li>
    <li>Check for partial autocorrelation of the time series (P)</li>
    <li>Using ARIMA((AutoRegressive Integrated Moving Average) model to forecast number of passengers</li>
        <ul>
            <li>7.1 Estimation step using TD_ARIMAESTIMATE</li>
            <li>7.2 Extract residuals</li>
            <li>7.3 Create table PLOT_ESTIMATE for plotting</li>
            <li>7.4 Validation step using TD_ARIMAVALIDATE</li>
            <li>7.5 Extract residuals</li>
            <li>7.6 Create table PLOT_VALIDATE for plotting</li>
            <li>7.7 Forecast step using TD_ARIMAFORECAST</li>
            <li>7.8 Create table PLOT_FORECAST for plotting</li>
        </ul>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>This steps connects with Vantage. You might be prompted to enter the password.</p>

In [1]:
%connect local

Password: ···


Success: 'local' connection established and activated for user 'demo_user'


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Explore the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>The data is classified in date/time and the passengers travelling per month. Time series deals with 2 columns, one is temporal i.e. <b>Date</b> in this case and another is the value to be forecasted i.e. <b>Passengers</b>. Following is a sample of the dataset.</p>

In [2]:
SELECT TOP 5 * FROM airpassengers ORDER BY "Date";

,Date,Passengers,seriesID
1,1949-01-01 00:00:00.000000,112,FlyHigh
2,1949-02-01 00:00:00.000000,118,FlyHigh
3,1949-03-01 00:00:00.000000,132,FlyHigh
4,1949-04-01 00:00:00.000000,129,FlyHigh
5,1949-05-01 00:00:00.000000,121,FlyHigh


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the total passengers by month from 1949 to 1960.  To view the image, right click in the <b>IMAGE</b> column and select <b>View Image</b></p>

In [3]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS("Passengers"),CONTENT(REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('line'),
            LEGEND('upper left'),
            TITLE('Number of Passengers Travelling Monthly')
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph from the query that looks like follows:</p>
<img id="fig1" src="images/fig1.png" alt="Number of Passengers Travelling Monthly" width="400" />
<p style = 'font-size:16px;font-family:Arial'>This shows that the data has yearly cycles and an upward trend.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Check for Stationarity using Dickey-Fuller Test</b>
<br>
<b style = font-size:16px;font-family:Arial>What is Stationarity?</b>
<br>
<p style = 'font-size:16px;font-family:Arial'>Before applying any statistical model on a Time Series, the series has to be staionary, which means that, over different time periods,
<br>
a) It should have constant mean.
<br>
b) It should have constant variance or standard deviation.
<br>
c) Auto-covariance should not depend on time.
</p>
<p style = 'font-size:16px;font-family:Arial'> We can visually examine if the mean and variance are constant over different periods of time or not. Alternatively, Dickey-Fuller test is widely used to check for stationarity in the time series data.
<br>

<p style = 'font-size:16px;font-family:Arial'>The result of the Dickey-Fuller Test:
<br>
• <b>ACCEPT</b> means the null hypothesis is accepted. The process
is non-stationary and unit roots are present.
<br>
• <b>REJECT</b> means the null hypothesis is rejected. The process is
stationary, and no unit roots are present.</p>

In [4]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
    SERIES_SPEC(
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        ALGORITHM('TREND')
    )
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,FlyHigh,0,144,TREND ,-4.639196964346928,0.0008857695636920808,ACCEPT 


<p style = 'font-size:16px;font-family:Arial'>Looking at the right most column NULL_HYP, the series is not stationary as the NULL Hypothesis is accepted. This suggests that we need to make this series stationary.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Make series stationary using TD_DIFF</b>
<p style = 'font-size:16px;font-family:Arial'>TD_DIFF transforms a stationary, seasonal, or nonstationary time series into a differenced time series. We are going to create a new table which is differenced. Here, the parameter <b>LAG</b> is set to 12(12 units of time i.e. 12 months or 1 year) in order to consider the cyclical nature of the data. We'll check if the newly created table is stationary or not(Using Dickey-Fuller test). If not, we'll again use TD_DIFF recursively till the series becomes stationary. Here, repeating it 1 time makes the series stationary.</p>

In [5]:
DROP TABLE diff1_air;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'diff1_air' does not exist.

In [6]:
EXECUTE FUNCTION
COLUMNS(OUT_Passengers AS Passengers)
INTO VOLATILE ART(diff1_air)
TD_DIFF(
    SERIES_SPEC(
            TABLE_NAME(airpassengers),
            ROW_AXIS(TIMECODE("Date")),
            SERIES_ID("seriesID"),
            PAYLOAD(
                FIELDS("Passengers"),
                CONTENT(REAL)
            )
    ),
    FUNC_PARAMS(
          LAG(12),
          DIFFERENCES(4),
          SEASONAL_MULTIPLIER(0)
    )
);

Success: 96 rows affected

In [7]:
SELECT TOP 5 * FROM diff1_air ORDER BY ROW_I;

,seriesID,ROW_I,Passengers
1,FlyHigh,0,34
2,FlyHigh,1,-10
3,FlyHigh,2,100
4,FlyHigh,3,78
5,FlyHigh,4,150


<p style = 'font-size:16px;font-family:Arial'>In the above result, <b>ROW_I</b> is Row identifier of the ordered result sets and <b>OUT_Passengers</b> is Transformed magnitudes of differenced time series elements.

In [8]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
    SERIES_SPEC(
        TABLE_NAME(diff1_air),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        ALGORITHM('TREND')
    )
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,FlyHigh,0,96,TREND ,-3.1342412241637474,0.09831876940084541,REJECT 


<p style = 'font-size:16px;font-family:Arial'>Looking at the right most column NULL_HYP, the series is stationary as the NULL Hypothesis is rejected. Differencing the series once gave us stationary series. Hence the value of order of differentiation or <b>D = 1</b>. This value will be used further in ARIMA_ESTIMATE.</p>
<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Check for autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>TD_ACF calculates the autocorrelation or autocovariance of a time series. The autocorrelation and autocovariance show how the time series covaries or correlates with itself when delayed by a lag in time or space. Here we check autocorrelation with a maximum lag of 12 time steps.</p>

In [9]:
DROP TABLE ACFDemo;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'ACFDemo' does not exist.

In [10]:
EXECUTE FUNCTION
COLUMNS(OUT_Passengers AS Auto_Correlation)
INTO VOLATILE ART(ACFDemo)
TD_ACF(
    SERIES_SPEC(
        TABLE_NAME(diff1_air),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        MAXLAGS(12),
        UNBIASED(1),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        FUNC_TYPE(0),   -- Use 0 for autocorrelation, or 1 for autocovariance
        DEMEAN(1),
        QSTAT(0),
        ALPHA(0.1)
    )
);

Success: 13 rows affected

In [11]:
SELECT TOP 5 * FROM ACFDemo ORDER BY ROW_I;

,seriesID,ROW_I,Auto_Correlation,CONF_OFF_Passengers,CONF_LOW_Passengers,CONF_HI_Passengers
1,FlyHigh,0,1,0,1,1
2,FlyHigh,1,0.736849780698874,0.16787717031655583,0.5689726103823182,0.9047269510154298
3,FlyHigh,2,0.6092948162493547,0.2424587615405742,0.36683605470878045,0.8517535777899289
4,FlyHigh,3,0.41962593972958917,0.28233204707048015,0.13729389265910902,0.7019579868000694
5,FlyHigh,4,0.3221378707614005,0.29939364430127435,0.022744226460126138,0.6215315150626748


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Auto Correlation Plot.  To view the image, right click in the <b>IMAGE</b> column and select <b>View Image</b></p>

In [12]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(ACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(Auto_Correlation, CONF_OFF_Passengers),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph from the query that looks like follows:</p>
<img id="fig2" src="images/fig2.png" alt="Auto Correlation" width="400" />
<p style = 'font-size:16px;font-family:Arial'>To get the value of Moving Average or Q, we need the lag where the value from the ACF plot falls in the significant limit. Looking at the graph, the Auto-Correlation value at ROW_I = 4 is outside the confidence band but is very much closer to it. Hence we can say that the value of Moving Average or <b>Q = 4</b>.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Check for partial autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>The TD_PACF function provides insight as to whether the function being modeled is stationary or not. The partial auto correlations are used to measure the degree of correlation between time series sample points. Here we check partial autocorrelation with a maximum lag of 12 time steps.</p>

In [13]:
DROP TABLE PACFDemo;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'PACFDemo' does not exist.

In [14]:
EXECUTE FUNCTION
COLUMNS(OUT_Passengers AS Partial_Auto_Correlation)
INTO VOLATILE ART(PACFDemo)
TD_PACF (
    SERIES_SPEC(
        TABLE_NAME(diff1_air),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ) ,
    FUNC_PARAMS(
        MAXLAGS(12),
        UNBIASED(0),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        ALGORITHM(LEVINSON_DURBIN),
        ALPHA(0.1)
    )
);

Success: 13 rows affected

In [15]:
SELECT TOP 5 * FROM PACFDemo ORDER BY ROW_I;

,seriesID,ROW_I,Partial_Auto_Correlation,CONF_OFF_Passengers,CONF_LOW_Passengers,CONF_HI_Passengers
1,FlyHigh,0,1,0.16787717031655583,1,1
2,FlyHigh,1,0.736849780698874,0.16787717031655583,0.5689726103823182,0.9047269510154298
3,FlyHigh,2,0.145163261004824,0.16787717031655583,-0.02271390931173184,0.3130404313213798
4,FlyHigh,3,-0.1589643249713953,0.16787717031655583,-0.32684149528795114,0.008912845345160542
5,FlyHigh,4,0.036264086522446906,0.16787717031655583,-0.1316130837941089,0.20414125683900275


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Partial Auto Correlation Plot.  To view the image, right click in the <b>IMAGE</b> column and select <b>View Image</b></p>

In [16]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(PACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(Partial_Auto_Correlation, CONF_OFF_Passengers),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph from the query that looks like follows:</p>
<img id="fig3" src="images/fig3.png" alt="Partial Auto Correlation" width="400" />
<p style = 'font-size:16px;font-family:Arial'>To get the value of Auto Regressive lags or P, we need the lag where the value from the PACF plot falls in the significant limit. Looking at the graph, the Partial Auto-Correlation value at ROW_I = 2 falls in the significant limit. Hence we can say that the value of Auto Regressive lags or <b>P = 2</b>.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. Using ARIMA (AutoRegressive Integrated Moving Average) model to forecast number of passengers</b>
<p style = 'font-size:16px;font-family:Arial'>
ARIMA functions on VANTAGE run in the following order:
<br>
1. Run ARIMAESTIMATE function to get the coefficients for the ARIMA model.
<br>
2. <i>[Optional]</i> Run ARIMAVALIDATE function to validate the the "goodness of fit" of the ARIMA model, when
FIT_PERCENTAGE is not 100 in TD_ARIMAESTIMATE.
<br>
3. Run the ARIMAFORECAST function with input from step 1 or step 2 to forecast the future periods
beyond the last observed period.
</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.1 Estimation step using TD_ARIMAESTIMATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAESTIMATE function estimates the coefficients corresponding to an ARIMA model, and to fit a series with an existing ARIMA model. The function can also provide the "goodness of fit" and the residuals of the fitting operation. The function generates model layer used as input for the TD_ARIMAVALIDATE and TD_ARIMAFORECAST functions. This function is for
univariate series.</p>

<br>

<p style = 'font-size:16px;font-family:Arial'>Here, the previously estimated parameters P, d and Q need to be passed in MODEL_ORDER(P, d, Q) i.e <b>MODEL_ORDER(2, 1, 4)</b>.</p>

In [17]:
DROP TABLE ART_EST;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'ART_EST' does not exist.

In [18]:
EXECUTE FUNCTION INTO VOLATILE ART(ART_EST)
TD_ARIMAESTIMATE(
    SERIES_SPEC(
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL))),
     FUNC_PARAMS(
        NONSEASONAL(MODEL_ORDER(2, 1, 4)),
        SEASONAL(MODEL_ORDER(2, 1, 4), PERIOD(12)),
        CONSTANT(1), COEFF_STATS(1), FIT_METRICS(1),
        RESIDUALS(1), ALGORITHM(CSS_MLE),  FIT_PERCENTAGE(80)
    )
);

Success: 162 rows affected

WARNING: [Teradata Database] [Warning 9155] UAF Function Warning: BFGS approximation may not converge.

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.2 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table output of previous estimation step.
</p>

In [19]:
DROP TABLE AR_RESIDUALS;

Success: 19 rows affected

In [20]:
CREATE TABLE AR_RESIDUALS AS (
    EXECUTE FUNCTION
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(ART_EST),
            LAYER(ARTFITRESIDUALS)
        )
    )
) WITH DATA;

Success: 0 rows affected

In [21]:
SELECT TOP 5 * FROM AR_RESIDUALS ORDER BY ROW_I;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,FlyHigh,0,112,111.93533678676759,0.06466321323240229
2,FlyHigh,1,118,117.96643416455044,0.03356583544955892
3,FlyHigh,2,132,131.96619387222813,0.03380612777188061
4,FlyHigh,3,129,128.97744817037776,0.022551829622242343
5,FlyHigh,4,121,120.98924613644182,0.010753863558178086


In [22]:
SELECT * FROM (
    EXECUTE FUNCTION
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(ART_EST),
            LAYER(ARTFITMETADATA)
        )
    )
) AS T;

,seriesID,ROW_I,NUM_SAMPLES,VAR_COUNT,R_SQUARE,R_ADJ_SQUARE,STD_ERROR,STD_ERROR_DF,ME,MAE,MSE,MPE,MAPE,FSTAT_CALC,P_VALUE,NUM_DF,DENOM_DF,SIGNIFICANCE_LEVEL,F_CRITICAL,F_CRITICAL_P,NULL_HYPOTH
1,FlyHigh,0,115,12,0.9932686139847002,0.9925497280995711,57.122421445513226,113,0.059099082677418806,5.483633988665126,56.12898802906952,0.023176837323505124,2.3222258837943666,1381.6777245617022,1.6589881514537377e-106,11,103,0.05,1.882814871188515,0.05,REJECT


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.3 Create table PLOT_ESTIMATE for plotting</b></p>
</p>

In [23]:
DROP TABLE PLOT_ESTIMATE;

Success: 17 rows affected

In [24]:
CREATE TABLE PLOT_ESTIMATE (DatasetID VARCHAR(10), ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [25]:
INSERT INTO PLOT_ESTIMATE SELECT 'FlyHigh', ROW_I, ACTUAL_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 
INSERT INTO PLOT_ESTIMATE SELECT 'ESTIMATED', ROW_I, CALC_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 

Success: 0 rows affected

Success: 226 rows affected

In [26]:
SELECT TOP 5 * FROM PLOT_ESTIMATE ORDER BY ROW_I;

,DatasetID,ROW_I,FIT_MAGNITUDE
1,ESTIMATED,2,131.96619387222813
2,FlyHigh,2,132
3,ESTIMATED,3,128.97744817037776
4,FlyHigh,3,129
5,FlyHigh,4,121


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Actual and Estimated values by the fitted ARIMA model.  To view the image, right click in the <b>IMAGE</b> column and select <b>View Image</b></p>

In [27]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(
        TABLE_NAME(PLOT_ESTIMATE),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(DataSetID),
        ID_SEQUENCE('[{"DatasetID":"FlyHigh"},{"DatasetID":"ESTIMATED"}]'),
        PAYLOAD(
            FIELDS(FIT_MAGNITUDE),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS
    (
        WIDTH(1920),
        HEIGHT(1080),
        TITLE('ARIMA ESTIMATE'),
        PLOTS[
            (
                TITLE ('ORIGINAL and ESTIMATED SERIES'),
                GRID(FORMAT('-')),
                TYPE('line'),
                SERIES[
                       (
                        ID(1),
                        FORMAT('r--')
                       ),
                       (
                        ID(2),
                        FORMAT('b-')
                       )
                     ],
                MARKER('o'),
                LEGEND('best'),
                XLABEL('X SeqNo'),
                YLABEL('Y Magnitude')
            )
        ]
    )
);

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph from the query that looks like follows:</p>
<img id="fig4" src="images/fig4.png" alt="ARIMA Estimate" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The red line is the actual number of passengers travelled and blue line is the estimated number of passengers travelled. This graph shows how well the ARIMA model has learnt on the training.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.4 Validation step using TD_ARIMAVALIDATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAVALIDATE function provides data scientists with a metrics collection for model selection and the produced residuals, such that a number of model validation tests can be performed. The TD_ARIMAVALIDATE function performs in-sample forecasting for both seasonal and non-seasonal auto-regressive (AR), moving-average (MA), and ARIMA models. It also supports the extended BoxJenkins seasonal ARIMA model formula.
</p>
<p style = 'font-size:16px;font-family:Arial'>We'll use the output of previous estimation step to validate the model. The testing train-validate split here for the dataset is 80:20. Hence 20% of data will be used for validation of the estimated model.</p>

In [28]:
DROP TABLE AR_VALIDATE;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'AR_VALIDATE' does not exist.

In [29]:
EXECUTE FUNCTION 
INTO VOLATILE ART(AR_VALIDATE)
TD_ARIMAVALIDATE(
    ART_SPEC(TABLE_NAME(ART_EST)),
    FUNC_PARAMS(
        FIT_METRICS(1),
        RESIDUALS(1)
    )
);

Success: 36 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.5 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table ouput of previous validation step.
</p>

In [30]:
DROP TABLE AR_VALIDATE_RESIDUALS;

Success: 19 rows affected

In [31]:
CREATE TABLE AR_VALIDATE_RESIDUALS AS (
    EXECUTE FUNCTION 
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(AR_VALIDATE),
            LAYER(ARTFITRESIDUALS)
        )
    )
) WITH DATA;

Success: 0 rows affected

In [32]:
SELECT TOP 5 * FROM AR_VALIDATE_RESIDUALS ORDER BY ROW_I;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,FlyHigh,102,505,504.71163108109863,0.28836891890137123
2,FlyHigh,103,404,403.94829126908496,0.05170873091501345
3,FlyHigh,104,359,359.00500927545085,-0.005009275450839275
4,FlyHigh,105,310,310.0458722149614,-0.04587221496140166
5,FlyHigh,106,337,337.0173333914181,-0.017333391418120594


In [33]:
SELECT * FROM (
    EXECUTE FUNCTION
    TD_EXTRACT_RESULTS(
        ART_SPEC(
            TABLE_NAME(AR_VALIDATE),
            LAYER(ARTFITMETADATA)
        )
    )
) AS T;

,seriesID,ROW_I,NUM_SAMPLES,VAR_COUNT,R_SQUARE,R_ADJ_SQUARE,STD_ERROR,STD_ERROR_DF,ME,MAE,MSE,MPE,MAPE,F_STAT,F_STAT_P,NUM_DF,DENOM_DF,SIGNIFICANCE_LEVEL,F_CRITICAL,F_CRITICAL_P,NULL_HYPOTH
1,FlyHigh,0,29,12,0.979248500671866,0.9658210599301322,141.4820936326631,27,-0.8362757746214149,6.8929387502658885,131.72470786489322,-0.20111313943644174,1.5210369103666177,72.92890130792145,4.540645283388951e-12,11,17,0.05,2.412561441820178,0.05,REJECT翾


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.6 Create table PLOT_VALIDATE for plotting</b></p>
</p>

In [34]:
DROP TABLE PLOT_VALIDATE;

Success: 17 rows affected

In [35]:
CREATE TABLE PLOT_VALIDATE (DatasetID VARCHAR(10), ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [36]:
INSERT INTO PLOT_VALIDATE SELECT 'FlyHigh', ROW_I, ACTUAL_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0; 
INSERT INTO PLOT_VALIDATE SELECT 'PREDICTED', ROW_I, CALC_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0;

Success: 0 rows affected

Success: 58 rows affected

<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Actual and Predicted values by ARIMA model. To view the image, right click in the <b>IMAGE</b> column and select <b>View Image</b></p>

In [37]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(
        TABLE_NAME(PLOT_VALIDATE),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(DataSetID),
        ID_SEQUENCE('[{"DatasetID":"FlyHigh"},{"DatasetID":"PREDICTED"}]'),
        PAYLOAD(
            FIELDS(FIT_MAGNITUDE),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS
    (
        WIDTH(1920),
        HEIGHT(1080),
        TITLE('ARIMA VALIDATE'),
        PLOTS[
            (
                TITLE ('ORIGINAL and PREDICTED SERIES'),
                GRID(FORMAT('-')),
                TYPE('line'),
                SERIES[
                       (
                        ID(1),
                        FORMAT('r--')
                       ),
                       (
                        ID(2),
                        FORMAT('b-')
                       )
                     ],
                MARKER('o'),
                LEGEND('best'),
                XLABEL('X SeqNo'),
                YLABEL('Y Magnitude')
            )
        ]
    )
);

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph from the query that looks like follows:</p>
<img id="fig5" src="images/fig5.png" alt="ARIMA Validate" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The red line is the actual number of passengers travelled and blue line is the predicted number of passengers travelled. This graph shows how well the ARIMA model predicts on testing data i.e. unseen data.</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.7 Forecast step using TD_ARIMAFORECAST</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAFORECAST function is used to forecast a user-defined number of periods based on
models fitted from the TD_ARIMAESTIMATE function.</p>
<p style = 'font-size:16px;font-family:Arial'>Here in the following cell, we use the estimated and validated model to forecast number of passengers for next 6 periods i.e. next 6 months.</p>

In [38]:
DROP TABLE ARMA_FORECAST;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'ARMA_FORECAST' does not exist.

In [39]:
EXECUTE FUNCTION INTO VOLATILE ART(ARMA_FORECAST)
TD_ARIMAFORECAST(
           ART_SPEC(TABLE_NAME(AR_VALIDATE)),
           FUNC_PARAMS(FORECAST_PERIODS(6)));

Success: 6 rows affected

In [40]:
SELECT * FROM ARMA_FORECAST;

,seriesID,ROW_I,FORECAST_VALUE,LO_80,HI_80,LO_95,HI_95
1,FlyHigh,0,444.961200759022,433.1503417874523,456.7720597305917,426.8980567013844,463.0243448166596
2,FlyHigh,1,416.3654215752107,401.45854101420065,431.27230213622073,393.56732266710594,439.16352048331544
3,FlyHigh,2,461.43561619162756,443.3817376498616,479.48949473339354,433.8246007960495,489.04663158720564
4,FlyHigh,3,495.5089129326341,475.9552189391356,515.0626069261325,465.604128503853,525.4136973614152
5,FlyHigh,4,508.42699487614186,488.01666861163,528.8373211406538,477.2121048781568,539.641884874127
6,FlyHigh,5,576.7516922723657,555.1667189489624,598.3366655957691,543.7403352187066,609.7630493260249


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>7.8 Create table PLOT_FORECAST for plotting</b></p>

In [41]:
DROP TABLE PLOT_FORECAST;

Success: 17 rows affected

In [42]:
CREATE TABLE PLOT_FORECAST (DatasetID VARCHAR(16), ROW_I BIGINT, FORECAST_MAGNITUDE FLOAT);

Success: 0 rows affected

In [43]:
INSERT INTO PLOT_FORECAST   SELECT 'FORECASTED', ROW_I, FORECAST_VALUE FROM ARMA_FORECAST; 
INSERT INTO PLOT_FORECAST   SELECT 'UPPER_BOUND', ROW_I, HI_80 FROM ARMA_FORECAST ; 
INSERT INTO PLOT_FORECAST   SELECT 'LOWER_BOUND', ROW_I, LO_80 FROM ARMA_FORECAST ; 

Success: 0 rows affected

Success: 0 rows affected

Success: 18 rows affected

In [44]:
SELECT * FROM PLOT_FORECAST ORDER BY ROW_I;

,DatasetID,ROW_I,FORECAST_MAGNITUDE
1,UPPER_BOUND,0,456.7720597305917
2,LOWER_BOUND,0,433.1503417874523
3,FORECASTED,0,444.961200759022
4,UPPER_BOUND,1,431.27230213622073
5,LOWER_BOUND,1,401.45854101420065
6,FORECASTED,1,416.3654215752107
7,UPPER_BOUND,2,479.48949473339354
8,LOWER_BOUND,2,443.3817376498616
9,FORECASTED,2,461.43561619162756
10,UPPER_BOUND,3,515.0626069261325


<p style = 'font-size:16px;font-family:Arial'>The <b>TD_PLOT</b> function will return an image in the cell of the results showing the Forecasted values by ARIMA model. To view the image, right click in the <b>IMAGE</b> column and select <b>View Image</b></p>

In [45]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(
        TABLE_NAME(PLOT_ARMA_FORECAST),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(DataSetID),
        ID_SEQUENCE('[{"DatasetID":"FORECASTED"},{"DatasetID":"UPPER_BOUND"},{"DatasetID":"LOWER_BOUND"}]'),
        PAYLOAD(
            FIELDS(FORECAST_MAGNITUDE),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS
    (
        WIDTH(1920),
        HEIGHT(1080),
        TITLE('ARIMA FORECAST'),
        PLOTS[
            (
                TITLE ('Forecast'),
                GRID(FORMAT('-')),
                TYPE('line'),
                SERIES[
                       (
                        ID(1),
                        FORMAT('r--')
                       ),
                       (
                        ID(2),
                        FORMAT('b-')
                       ),
                        (
                        ID(3),
                        FORMAT('b-')
                       )
                     ],
                MARKER('o'),
                LEGEND('best'),
                XLABEL('X SeqNo'),
                YLABEL('Y Magnitude')
            )
        ]
    )
);

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<p style = 'font-size:16px;font-family:Arial'>If you followed the instructions above, you should have seen a graph from the query that looks like follows:</p>
<img id="fig6" src="images/fig6.png" alt="ARIMA Forecast" width="400" />
<p style = 'font-size:16px;font-family:Arial'>The red line is the actual number of passengers travelled and blue lines are the upper and lower confidence interval with 80% confidence level. The red line is the Forecasted number of passengers in the next 6 months.</p>

<p style = 'font-size:16px;font-family:Arial'>We have trained and validated the ARIMA model on the air passengers dataset and the results match closely with the actual data. The goodness of fit metrics calculated in estimate and validate phase also resonate with our understanding that the model is well trained in order to forecast.</p>
<footer style="paddinga:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2022 Teradata. All Rights Reserved</footer>